In [19]:
import requests
import pandas as pd

# Fetch data from the API for training (1 year) and testing (1 month)
url_train = "https://api.energy-charts.info/price?bzn=DE-LU&start=2024-01-01&end=2024-09-01"
url_test = "https://api.energy-charts.info/price?bzn=DE-LU&start=2024-09-01&end=2024-10-01"

response_train = requests.get(url_train)
response_test = requests.get(url_test)

data_train = response_train.json()
data_test = response_test.json()

# Convert to DataFrame
train_df = pd.DataFrame({
    'timestamp': pd.to_datetime(data_train['unix_seconds'], unit='s'),
    'price': data_train['price']
})
test_df = pd.DataFrame({
    'timestamp': pd.to_datetime(data_test['unix_seconds'], unit='s'),
    'price': data_test['price']
})

# Clean the data (remove rows with null prices)
train_df = train_df.dropna()
test_df = test_df.dropna()

In [ ]:
test_df.head(100)

In [ ]:
train_df.head(1000)

In [ ]:
train_df = train_df.rename(columns={'timestamp': 'ds', 'price': 'y'})
test_df = test_df.rename(columns={'timestamp': 'ds', 'price': 'y'})
train_df['unique_id'] = 'electricity_prices'
test_df['unique_id'] = 'electricity_prices'

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS

horizon = len(test_df)

models = [
    LSTM(h=horizon,  # Forecast horizon (predict the next day's price)
         max_steps=500,  # Number of steps to train
         scaler_type='standard',  # Type of scaler
         encoder_hidden_size=64,  # LSTM encoder hidden size
         decoder_hidden_size=64),  # LSTM decoder hidden size
    NHITS(h=horizon,  # Forecast horizon
          input_size=2 * horizon,  # Input sequence length (using data from previous 2 days)
          max_steps=100,  # Number of steps to train
          n_freq_downsample=[2, 1, 1])  # Downsampling factors for the stack output
]

In [ ]:
nf = NeuralForecast(models=models, freq='H')  # Hourly data (adjust according to your data frequency)
nf.fit(df=train_df)

In [ ]:
Y_hat_df = nf.predict()
Y_hat_df = Y_hat_df.reset_index()

In [ ]:
Y_hat_df.head(50)  # Check predictions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure 'ds' columns are in datetime format
test_df['ds'] = pd.to_datetime(test_df['ds'])
Y_hat_df['ds'] = pd.to_datetime(Y_hat_df['ds'])

# Print the last date in train_df and first date in Y_hat_df to verify continuity
print("Last date in test_df:", test_df['ds'].max())
print("First date in Y_hat_df:", Y_hat_df['ds'].min())

# If dates are continuous, proceed to plot
# Concatenate actual and forecasted data
plot_df = pd.concat([
    test_df[['ds', 'y']].set_index('ds'),
    Y_hat_df[['ds', 'LSTM', 'NHITS']].set_index('ds')
], axis=1)

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(20, 7))
plot_df[['y', 'LSTM', 'NHITS']].plot(ax=ax, linewidth=2)

ax.set_title('Electricity Price Forecast', fontsize=22)
ax.set_ylabel('Price', fontsize=20)
ax.set_xlabel('Date', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()
plt.show()